# TIC-TAC-TOE Markov Decision Process

In [1]:
%run tic_tac_toe_aux_funcs.py

## States
- A board is represented with a 3x3 matrix and each cell can be filled with *None*, X or O, numerically represented with 0, 1 and 2. Hence, the number of possible boards is 3^9 = 19683.
- Each board is uniquely identified with an ID, obtained by the conversion in base-10 of the flattened board.
- Many boards aren't valid (e.g. the number of Xs minus number of Os is less than 0 or greater than 1) and many are equal (e.g. flip and rotation).
- Only valid and unique *afterstates* are considered states of the MDP, plus three *terminal states* to model *draw*, *win* and *loss* for every board configuration that would lead to these situations. Hence, the number of states is much lower than 3^9.
- To map and speed up the relations between IDs and states two lookup tables will be created:
    - **id_to_state** -> IDs are associated to the following informations:
        1. The type of the board: -1 not valid, 1 terminal, 0 otherwise. 
        2. If it's terminal the winner, otherwise the next player.
        3. If it's necessary to flip this board to get the unique one.
        4. If it's necessary to rotate this board to get the unique one and how many times.
        5. The corresponding state.
    - **state_to_id** -> States are associated to the following informations:
        1. The type of the board: -1 not valid, 1 terminal, 0 otherwise. 
        2. If it's terminal the winner, otherwise the next player.
        3. The smallest associated board ID, i.e. the corresponding unique board.

In [2]:
# Number of boards<->IDs to check.
IDs = 3 ** 9
# Create the two lookup tables.
id_state_lkt = np.zeros((IDs, 5), dtype=np.int32)
state_id_lkt = np.empty([0, 3], dtype=np.int32)
# Loop on IDs.
state = 0
for i in range(IDs):
    # Convert the ID to the conrresponding board.
    board = id_to_board(i)
    # Compute the board rotations and symmetries
    # (this will be an info matrix).
    ids = board_to_ids(board)
    # Skip this board if already evaluated, i.e. we already
    # processed the unique corresponding one that will have a lower ID.
    if ids[0, 0] < i:
        continue
    # Get board info.
    info = board_info(board)
    # Check if the board is valid.
    if info[0] == -1:
        # Invalid board.
        # Populate the id_to_state lookup table.
        for id in ids:
            id_state_lkt[id[0], :] = [-1, -1, -1, -1, -1]
    elif info[0] == 0:
        # Non-terminal board.
        # Check next player.
        if info[1] == 1:
            # X's turn: populate the id_to_state lookup table.
            for id in ids:
                id_state_lkt[id[0], :] = [info[0], info[1], id[1], id[2], state]
            # Populate the state_to_id lookup table.
            state_id_lkt = np.append(state_id_lkt, [[info[0], info[1], ids[0, 0]]], axis=0)
            # Increase the state counter.
            state += 1
        else:
            # O's turn: not a state.
            for id in ids:
                id_state_lkt[id[0], :] = [info[0], info[1], id[1], id[2], -1]
    else:
        # Terminal board.
        # We place dummy values that we'll replace.
        if info[1] == 0:
            # Draw: -2.
            for id in ids:
                id_state_lkt[id[0], :] = [info[0], info[1], id[1], id[2], -2]
        elif info[1] == 1:
            # Win: -3.
            for id in ids:
                id_state_lkt[id[0], :] = [info[0], info[1], id[1], id[2], -3]
        else:
            # Loss: -4.
            for id in ids:
                id_state_lkt[id[0], :] = [info[0], info[1], id[1], id[2], -4]
# Add terminal states to lookup table.
state_id_lkt = np.append(state_id_lkt, [[1, 0, -1]], axis=0)  # index: state
state_id_lkt = np.append(state_id_lkt, [[1, 1, -2]], axis=0)  # index: state + 1
state_id_lkt = np.append(state_id_lkt, [[1, 2, -3]], axis=0)  # index: state + 2
# Remap terminal states.
id_state_lkt[(id_state_lkt[:, -1] == -2), -1] = state
id_state_lkt[(id_state_lkt[:, -1] == -3), -1] = state + 1
id_state_lkt[(id_state_lkt[:, -1] == -4), -1] = state + 2

# Number of states, now only afterstates plus three.
S = state_id_lkt.shape[0]
# Number of actions.
A = 9

## Transitions matrix
- The transitions matrix P has dimensions *S x S x A*.
- We directly compute each entry of the matrix, which is the probability of getting from state *s* to state *s_p* taking an action *a*:
    - At first, if the current state is terminal all actions should lead here, so P(s, s, :) = 1.
    - If the state is not terminal we have to check each action and tell if it can be done here or not:
        - By choice, *infeasible* actions keep the player in the current state, so again P(s, s, a) = 1. We'll deal with this in the reward matrix.
        - If the actions is valid, we compute the resulting board, the feasible actions for the opponent, and for each non-terminal state we set uniform probability.

In [3]:
prec = np.float128

# Create transitions matrix P.
P = np.zeros((S, S, A), dtype=prec)
for s in range(S):
    info = state_id_lkt[s]
    if info[0] == 1:
        # Terminal state: all actions lead here.
        P[s, s, :] = 1.0
    else:
        # Not a terminal board.
        # What can we do?
        actions_X = get_actions(info[2])
        # SANITY CHECK
        if len(actions_X) == 0:
            print("ERROR IN P GENERATION: NO POSSIBLE ACTIONS FOR X.")
            raise
        for a_X in range(A):
            if a_X in actions_X:
                # Possible action.
                # Compute the new board and its ID.
                id = np.copy(info[2])
                id_X = id + (3 ** (8 - a_X))
                # Check if this is a terminal state.
                if id_state_lkt[id_X, 0] == 1:
                    if id_state_lkt[id_X, 1] == 0:
                        # Draw.
                        P[s, S - 3, a_X] = 1.0
                    elif id_state_lkt[id_X, 1] == 1:
                        # Win.
                        P[s, S - 2, a_X] = 1.0
                    else:
                        # This is impossible beacuse there cannot be a defeat for X if it
                        # has just played.
                        print("ERROR IN P GENERATION: TERMINAL STATE INVALID.")
                        raise
                else:
                    # Get the next state from the new ID.
                    actions_O = get_actions(id_X)
                    if len(actions_O) == 0:
                        print("ERROR IN P GENERATION: EMPTY O ACTIONS.")
                        raise
                    for a_O in actions_O:
                        id_O = id_X + 2 * (3 ** (8 - a_O))
                        sp = id_state_lkt[id_O, 4]
                        # SANITY CHECK
                        if sp < 0:
                            print("ERROR IN P GENERATION: NEXT STATE {} INVALID.".format([id_O, sp]))
                            raise
                        P[s, sp, a_X] += 1.0 / float(len(actions_O))
            else:
                # Impossible action: stay here.
                P[s, s, a_X] = 1.0

## Row-stochasticity checks
P has to be row-stochastic for this to work.

In [4]:
sto = np.zeros((S, A))
for a in range(A):
    for i, row in enumerate(P[:, :, a]):
        sto[i, a] = np.sum(row)
        if sto[i, a] != 1.0:
            print("ERROR IN P GENERATION: ROW-STOCHASTICITY VIOLATED IN (ROW, ACTION) {}.".format((i, a)))
            raise
print("P is row-stochastic.")

P is row-stochastic.


## Rewards matrix
- Expected rewards matrix R has dimentions *S x A*.
- +1 for every (state, action) pair that corresponds to a victory.
- -1 for every (state, action) pair that corresponds to a defeat.
- For every other pair we can choose between:
    - Assigning 0: this will result in the optimal policy taking unnecessary actions that will in any case result in a victory or a draw, although not so quickly and often in multiple ways at the same time.
    - Assigning -1: this will result in the optimal policy being time-greedy, taking the least possible number of actions to win.
- **To make this work we need a _soft constraint_: all infeasible (state, action) pairs have a negative reward, e.g. -1.**

We create this matrix doing as follows:
- Create a reward vector of size S that for each state holds 1, 0 or -1 depending on its type.
- For each (state, action) pair, take the transition probability vector P(s, :, a).
- Compute the expected reward for this pair as the inner product of the two former vectors.


In [5]:
prec = np.float128
# Create the rewards vector.
# Set this for standard reward.
reward_vector = np.zeros(S, dtype=prec)
reward_vector[-2] = 1.0
reward_vector[-1] = -1.0
# Set this for least possible number of moves.
# reward_vector = np.full(S, -1.0, dtype=prec)
# reward_vector[-2] = 10.0

# Create rewards matrix.
R = np.zeros((S, A), dtype=prec)
# Loop on states.
for s in range(S):
    # Get state info.
    info = state_id_lkt[s]
    if info[0] == 1:
        continue
    # Get valid actions.
    actions = get_actions(info[2])
    # Loop on actions.
    for a in range(A):
        if a in actions:
            R[s, a] = np.matmul(np.copy(P[s, :, a]), reward_vector, dtype=prec)
        else:
            # Soft constrain for infeasible moves.
            R[s, a] = -10.0

In [6]:
# Dump data on files.
id_state_lkt.dump("ttt_id2s.dat")
state_id_lkt.dump("ttt_s2id.dat")
P.dump("ttt_P.dat")
R.dump("ttt_R.dat")